In [221]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
import re
import joblib

In [222]:

#multiple linear regression
car=pd.read_csv("D:\ML projects\car_price _prediction\Cardetails.csv")

print(car.head())

                           name  year  selling_price  km_driven    fuel  \
0        Maruti Swift Dzire VDI  2014         450000     145500  Diesel   
1  Skoda Rapid 1.5 TDI Ambition  2014         370000     120000  Diesel   
2      Honda City 2017-2020 EXi  2006         158000     140000  Petrol   
3        Hyundai  Sportz Diesel  2010         225000     127000  Diesel   
4        Maruti Swift VXI BSIII  2007         130000     120000  Petrol   

  seller_type transmission         owner     mileage   engine   max_power  \
0  Individual       Manual   First Owner   23.4 kmpl  1248 CC      74 bhp   
1  Individual       Manual  Second Owner  21.14 kmpl  1498 CC  103.52 bhp   
2  Individual       Manual   Third Owner   17.7 kmpl  1497 CC      78 bhp   
3  Individual       Manual   First Owner   23.0 kmpl  1396 CC      90 bhp   
4  Individual       Manual   First Owner   16.1 kmpl  1298 CC    88.2 bhp   

                     torque  seats  
0            190Nm@ 2000rpm    5.0  
1       250N

In [223]:
brandlist = []
for i in car['name']:
    #print(i,"  ")
    sentence = i.split(" ")
    brand = sentence[0]
    brandlist.append(brand)

In [224]:
car['Brand'] = brandlist

In [225]:
x1=car.drop(["selling_price","name","seller_type","owner","torque"],axis='columns')
print(x1.head())

   year  km_driven    fuel transmission     mileage   engine   max_power  \
0  2014     145500  Diesel       Manual   23.4 kmpl  1248 CC      74 bhp   
1  2014     120000  Diesel       Manual  21.14 kmpl  1498 CC  103.52 bhp   
2  2006     140000  Petrol       Manual   17.7 kmpl  1497 CC      78 bhp   
3  2010     127000  Diesel       Manual   23.0 kmpl  1396 CC      90 bhp   
4  2007     120000  Petrol       Manual   16.1 kmpl  1298 CC    88.2 bhp   

   seats    Brand  
0    5.0   Maruti  
1    5.0    Skoda  
2    5.0    Honda  
3    5.0  Hyundai  
4    5.0   Maruti  


In [226]:
print(x1.shape)

(8128, 9)


In [227]:
y=car.selling_price
print(y.head())
y

0    450000
1    370000
2    158000
3    225000
4    130000
Name: selling_price, dtype: int64


0       450000
1       370000
2       158000
3       225000
4       130000
         ...  
8123    320000
8124    135000
8125    382000
8126    290000
8127    290000
Name: selling_price, Length: 8128, dtype: int64

In [228]:
x1["fuel"].unique()

array(['Diesel', 'Petrol', 'LPG', 'CNG'], dtype=object)

In [229]:
x1["transmission"].unique()


array(['Manual', 'Automatic'], dtype=object)

In [230]:
kk = x1["Brand"].unique()
kk

array(['Maruti', 'Skoda', 'Honda', 'Hyundai', 'Toyota', 'Ford', 'Renault',
       'Mahindra', 'Tata', 'Chevrolet', 'Fiat', 'Datsun', 'Jeep',
       'Mercedes-Benz', 'Mitsubishi', 'Audi', 'Volkswagen', 'BMW',
       'Nissan', 'Lexus', 'Jaguar', 'Land', 'MG', 'Volvo', 'Daewoo',
       'Kia', 'Force', 'Ambassador', 'Ashok', 'Isuzu', 'Opel', 'Peugeot'],
      dtype=object)

In [231]:
no_of_brands = kk.size
no_of_brands

32

In [232]:
dictkeys = kk
keys = list(dictkeys)
values = list(range(0,no_of_brands))
dictionary = dict(zip(keys, values))
print(dictionary) 

{'Maruti': 0, 'Skoda': 1, 'Honda': 2, 'Hyundai': 3, 'Toyota': 4, 'Ford': 5, 'Renault': 6, 'Mahindra': 7, 'Tata': 8, 'Chevrolet': 9, 'Fiat': 10, 'Datsun': 11, 'Jeep': 12, 'Mercedes-Benz': 13, 'Mitsubishi': 14, 'Audi': 15, 'Volkswagen': 16, 'BMW': 17, 'Nissan': 18, 'Lexus': 19, 'Jaguar': 20, 'Land': 21, 'MG': 22, 'Volvo': 23, 'Daewoo': 24, 'Kia': 25, 'Force': 26, 'Ambassador': 27, 'Ashok': 28, 'Isuzu': 29, 'Opel': 30, 'Peugeot': 31}


In [233]:


data = {'mileage': car["mileage"], 'engine': car['engine'], 'max_power': car['max_power']}

car = pd.DataFrame(data)
x2 = pd.DataFrame()
x2 = x1.drop(['fuel','transmission'],axis =1)
x2['mileage'] = x1['mileage'].str.extract(r'(\d+(?:\.\d+)?)')
x2['engine'] = x1['engine'].str.extract(r'(\d+(?:\.\d+)?)')
x2['max_power'] = x1['max_power'].str.extract(r'(\d+(?:\.\d+)?)')
x2.loc[: ,'transmission']=x1.transmission.map({"Manual":0,"Automatic":1})
x2.loc[: ,'fuel']=x1.fuel.map({"Diesel":0,"Petrol":1,"LPG":2,"CNG":3})
x2.loc[: ,'Brand']=x1.Brand.map(dictionary)



x1
x3 = pd.DataFrame()
x3 = x2.fillna('0',inplace = False)
x3

,year,km_driven,mileage,engine,max_power,seats,Brand,transmission,fuel
0,2014,145500,23.4,1248,74,5.0,0,0,0
1,2014,120000,21.14,1498,103.52,5.0,1,0,0
2,2006,140000,17.7,1497,78,5.0,2,0,1
3,2010,127000,23.0,1396,90,5.0,3,0,0
4,2007,120000,16.1,1298,88.2,5.0,0,0,1
...,...,...,...,...,...,...,...,...,...
8123,2013,110000,18.5,1197,82.85,5.0,3,0,1
8124,2007,119000,16.8,1493,110,5.0,3,0,0
8125,2009,120000,19.3,1248,73.9,5.0,0,0,0
8126,2013,25000,23.57,1396,70,5.0,8,0,0


In [234]:
x3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8128 entries, 0 to 8127
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   year          8128 non-null   int64 
 1   km_driven     8128 non-null   int64 
 2   mileage       8128 non-null   object
 3   engine        8128 non-null   object
 4   max_power     8128 non-null   object
 5   seats         8128 non-null   object
 6   Brand         8128 non-null   int64 
 7   transmission  8128 non-null   int64 
 8   fuel          8128 non-null   int64 
dtypes: int64(5), object(4)
memory usage: 571.6+ KB


In [235]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x3,y,test_size=0.3,random_state=10)


In [236]:
reg=linear_model.LinearRegression()


In [237]:
reg.fit(x_train,y_train)


LinearRegression()

In [238]:
y_pred=reg.predict(x_test)
y_pred.reshape(-1,1)

array([[ 328404.11564618],
       [ 614434.17909178],
       [ 869458.24891463],
       ...,
       [ 884571.84615366],
       [1955096.63599825],
       [1240171.29639363]])

In [239]:
type(y_test)
y_test =np.array(y_test)
y_test.reshape(1,-1)
print(type(y_pred))
type(y_test)

<class 'numpy.ndarray'>


numpy.ndarray

In [240]:

r_squared=reg.score(x_train,y_train)
print(r_squared)

0.6709205414046289


In [241]:
r_squared=reg.score(x_test,y_test)
print(r_squared)

0.6643145237857069


In [242]:
from sklearn.metrics import r2_score
scores = []
for i in range(1000):
    x_train,x_test,y_train,y_test=train_test_split(x3,y,test_size=0.2,random_state=i)
    reg=linear_model.LinearRegression()
    reg.fit(x_train,y_train)
    y_pred=reg.predict(x_test)
    #r_squared=reg.score(x_test,y_test)
    scores.append(r2_score(y_test,y_pred))


In [243]:
scores[np.argmax(scores)]

0.7053505576672972

In [244]:
import pickle
pickle.dump(reg,open('car.pkl','wb'))